In [1]:
import keras_tuner
from tensorflow import keras
import tensorflow as tf
import os
import cv2
import numpy as np
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import pathlib
from keras.applications.densenet import DenseNet121
from keras.layers import BatchNormalization, Activation, Dropout, MaxPooling2D,Conv2D
import pandas as pd

2023-04-17 19:06:28.205376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 19:06:30.223231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
size = 96


In [3]:
#EmotioNet
EmotioNet_Emotion_X_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_X_train")
EmotioNet_Emotion_y_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_Y_train")
EmotioNet_Emotion_X_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_X_val")
EmotioNet_Emotion_y_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_Y_val")
EmotioNet_Emotion_X_test = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_X_test")
EmotioNet_Emotion_y_test = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_Y_test")
X_train = EmotioNet_Emotion_X_train
X_val = EmotioNet_Emotion_X_val
X_test = EmotioNet_Emotion_X_test
y_train = EmotioNet_Emotion_y_train
y_val = EmotioNet_Emotion_y_val
y_test = EmotioNet_Emotion_y_test

In [4]:
from skimage.transform import resize
def resize_images(images):
    resized_images = []
    for image in images:
        resized_images.append(resize(image, (size, size)))
    return np.asarray(resized_images).astype('float32')

In [5]:
X_train = resize_images(X_train)
X_val = resize_images(X_val)
X_test = resize_images(X_test)

In [6]:
X_train = np.resize(X_train,(len(X_train),size,size))
X_val = np.resize(X_val,(len(X_val),size,size))
X_test = np.resize(X_test,(len(X_test),size,size))

In [7]:
X_train = np.resize(X_train,(len(X_train),size,size,1))
X_val = np.resize(X_val,(len(X_val),size,size,1))
X_test = np.resize(X_test,(len(X_test),size,size,1))

In [8]:
print(np.shape(X_train))
print(np.shape(X_val))
print(np.shape(X_test))

(877, 96, 96, 1)
(110, 96, 96, 1)
(110, 96, 96, 1)


In [9]:
y_train

216    3.0
512    5.0
175    3.0
594    4.0
300    5.0
      ... 
743    2.0
263    3.0
786    5.0
462    5.0
669    5.0
Name: emotion, Length: 877, dtype: float64

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
y_train = np.array(np.array(y_train).reshape(-1, 1))
y_val = np.array(np.array(y_val).reshape(-1, 1))
y_test = np.array(np.array(y_test).reshape(-1, 1))

In [12]:
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder.fit_transform(y_train)
onehot_encoder2 = OneHotEncoder(sparse_output=False)
y_val = onehot_encoder2.fit_transform(y_val)
#y_test = onehot_encoder2.fit_transform(y_test)

In [13]:
np.shape(y_val)

(110, 6)

In [14]:
np.shape(y_train)

(877, 6)

In [15]:
NUM_EPOCHS = 20

In [16]:
def change_input_layer_dense(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False  #You can make this trainable if you want.
    updated_model.summary()
    return updated_model

In [17]:
densenet_model = DenseNet121(include_top=False, weights='imagenet')
densenet_updated = change_input_layer_dense(densenet_model, 2, size, size, 1)

2023-04-17 19:06:37.893460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-17 19:06:38.075222: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-17 19:06:38.076096: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

['input_1', 'zero_padding2d', 'conv1/conv', 'conv1/bn', 'conv1/relu', 'zero_padding2d_1', 'pool1', 'conv2_block1_0_bn', 'conv2_block1_0_relu', 'conv2_block1_1_conv', 'conv2_block1_1_bn', 'conv2_block1_1_relu', 'conv2_block1_2_conv', 'conv2_block1_concat', 'conv2_block2_0_bn', 'conv2_block2_0_relu', 'conv2_block2_1_conv', 'conv2_block2_1_bn', 'conv2_block2_1_relu', 'conv2_block2_2_conv', 'conv2_block2_concat', 'conv2_block3_0_bn', 'conv2_block3_0_relu', 'conv2_block3_1_conv', 'conv2_block3_1_bn', 'conv2_block3_1_relu', 'conv2_block3_2_conv', 'conv2_block3_concat', 'conv2_block4_0_bn', 'conv2_block4_0_relu', 'conv2_block4_1_conv', 'conv2_block4_1_bn', 'conv2_block4_1_relu', 'conv2_block4_2_conv', 'conv2_block4_concat', 'conv2_block5_0_bn', 'conv2_block5_0_relu', 'conv2_block5_1_conv', 'conv2_block5_1_bn', 'conv2_block5_1_relu', 'conv2_block5_2_conv', 'conv2_block5_concat', 'conv2_block6_0_bn', 'conv2_block6_0_relu', 'conv2_block6_1_conv', 'conv2_block6_1_bn', 'conv2_block6_1_relu', 'conv

In [18]:
num_classes = 6

In [19]:
precision = keras.metrics.Precision(name="Precision")
recall = keras.metrics.Recall(name="Recall")

In [20]:
from keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

auc_metric = tf.keras.metrics.AUC(
num_thresholds=200, curve="ROC",
summation_method="interpolation",
thresholds=None, multi_label=False
)


In [21]:
import datetime
folder_name = 'logs/DenseNet/Tuning/Emotion/'
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [22]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/Tuning/DenseNet',monitor='val_custom_f1', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [23]:
def build_model(hp):
    lrate = hp.Float('lrate', 1e-4, 1e-1, sampling='log')
    l1 = 0
    l2 = hp.Choice('l2',  values=[0.0, 1e-1, 1e-2, 1e-3, 1e-4])
    num_hidden = hp.Int('num_hidden', 32,1024,32)
    regularizer = tf.keras.regularizers.l1_l2(l1, l2)
    model = Sequential()
    model.add(densenet_updated)
    model.add(Flatten())
    model.add(Dense(num_hidden, kernel_regularizer=regularizer))
    model.add(BatchNormalization(scale=False,
                                    center=False))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.4))
    model.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss='categorical_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])
    return model


In [24]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective=keras_tuner.Objective('val_custom_f1', 'max'),
    max_trials = 20,
    overwrite=False,
    directory="./Emotion_Tuning/"
)

INFO:tensorflow:Reloading Tuner from ./Emotion_Tuning/untitled_project/tuner0.json


In [26]:
tuner.search(
    X_train, y_train, validation_data=(X_val, y_val),
    epochs=20
)

Trial 11 Complete [00h 00m 47s]
val_custom_f1: 0.7878770232200623

Best val_custom_f1 So Far: 0.8178023099899292
Total elapsed time: 00h 07m 02s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
0.0011766         |0.00036474        |lrate
0.0001            |0.0001            |l2
512               |704               |num_hidden

Epoch 1/20
28/28 [==============================] - 11s 165ms/step - loss: 1.4167 - Precision: 0.7180 - Recall: 0.5856 - custom_f1: 0.6228 - auc: 0.8657 - accuracy: 0.6214 - val_loss: 1.7062 - val_Precision: 0.7670 - val_Recall: 0.7182 - val_custom_f1: 0.7514 - val_auc: 0.9050 - val_accuracy: 0.7273
Epoch 2/20
28/28 [==============================] - 3s 100ms/step - loss: 0.5281 - Precision: 0.8937 - Recall: 0.8050 - custom_f1: 0.8482 - auc: 0.9781 - accuracy: 0.8495 - val_loss: 1.1886 - val_Precision: 0.7714 - val_Recall: 0.7364 - val_custom_f1: 0.7573 - val_auc: 0.9277 - val_accuracy: 0.7455
Epoch 3/20
28/28 [====================

KeyboardInterrupt: 

In [27]:
topN = 1
for x in range(topN):
    print(tuner.get_best_hyperparameters(topN)[x].values)
    print(tuner.get_best_models(topN)[x].summary())

{'lrate': 0.0003647363767459379, 'l2': 0.0001, 'num_hidden': 704}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 3, 3, 1024)        7031232   
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 704)               6488768   
                                                                 
 batch_normalization (BatchN  (None, 704)              1408      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 704)               0         
                                                                 
 dropout (Dropout)           (None, 704)               0

Training

In [21]:
regularizer = tf.keras.regularizers.l1_l2(0, 0.0001)
model = Sequential()
model.add(densenet_updated)
model.add(Flatten())
model.add(Dense(704, kernel_regularizer=regularizer))
model.add(BatchNormalization(scale=False,
                                center=False))
model.add(Activation('relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003647363767459379),
                loss='categorical_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])

In [22]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/Tuned_Emotion_EmotioNet',monitor='val_custom_f1', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./clean_logs/Emotion/Tuned_Model_EmotioNet")
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [23]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
    epochs=20, callbacks=callbacks)

Epoch 1/20


2023-04-17 19:07:21.511653: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-17 19:07:23.244821: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-17 19:07:26.500854: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2e9b8980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-17 19:07:26.500892: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 970M, Compute Capability 5.2
2023-04-17 19:07:26.550996: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-17 19:07:26.832644: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-17 19:07:26.914577: I ./tensorflow/compiler/jit/device_compiler

28/28 [==============================] - ETA: 0s - loss: 1.4311 - Precision: 0.7010 - Recall: 0.5428 - custom_f1: 0.6051 - auc: 0.8526 - accuracy: 0.5975

2023-04-17 19:07:52.507974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,704]
	 [[{{node inputs}}]]
2023-04-17 19:08:00.568145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,704]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: ./chkpts/Tuned_Emotion_EmotioNet/assets


...

28/28 [==============================] - 43s 2s/step - loss: 0.1371 - Precision: 0.9942 - Recall: 0.9852 - custom_f1: 0.9898 - auc: 0.9992 - accuracy: 0.9886 - val_loss: 0.9010 - val_Precision: 0.8286 - val_Recall: 0.7909 - val_custom_f1: 0.8065 - val_auc: 0.9432 - val_accuracy: 0.7909
Epoch 10/20
28/28 [==============================] - 2s 56ms/step - loss: 0.1307 - Precision: 0.9920 - Recall: 0.9863 - custom_f1: 0.9893 - auc: 0.9996 - accuracy: 0.9886 - val_loss: 0.9443 - val_Precision: 0.8333 - val_Recall: 0.7727 - val_custom_f1: 0.8035 - val_auc: 0.9435 - val_accuracy: 0.7909
Epoch 11/20
28/28 [==============================] - 2s 56ms/step - loss: 0.1271 - Precision: 0.9897 - Recall: 0.9852 - custom_f1: 0.9861 - auc: 0.9991 - accuracy: 0.9875 - val_loss: 1.1111 - val_Precision: 0.7981 - val_Recall: 0.7545 - val_custom_f1: 0.7747 - val_auc: 0.9334 - val_accuracy: 0.7727
Epoch 12/20
28/28 [==============================] - 2s 56ms/step - loss: 0.1172 - Precision: 0.9943 - Recall: 0.

2023-04-17 19:11:46.767770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,704]
	 [[{{node inputs}}]]
2023-04-17 19:11:53.867236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,704]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: ./chkpts/Tuned_Emotion_EmotioNet/assets


INFO:tensorflow:Assets written to: ./chkpts/Tuned_Emotion_EmotioNet/assets


28/28 [==============================] - 41s 2s/step - loss: 0.1054 - Precision: 0.9931 - Recall: 0.9897 - custom_f1: 0.9916 - auc: 0.9998 - accuracy: 0.9897 - val_loss: 1.0006 - val_Precision: 0.8350 - val_Recall: 0.7818 - val_custom_f1: 0.8085 - val_auc: 0.9486 - val_accuracy: 0.7818
Epoch 17/20
28/28 [==============================] - 2s 57ms/step - loss: 0.0970 - Precision: 0.9977 - Recall: 0.9966 - custom_f1: 0.9972 - auc: 0.9992 - accuracy: 0.9977 - val_loss: 0.9699 - val_Precision: 0.8252 - val_Recall: 0.7727 - val_custom_f1: 0.8005 - val_auc: 0.9534 - val_accuracy: 0.7818
Epoch 18/20
28/28 [==============================] - 2s 57ms/step - loss: 0.0926 - Precision: 0.9966 - Recall: 0.9943 - custom_f1: 0.9955 - auc: 0.9999 - accuracy: 0.9966 - val_loss: 0.8736 - val_Precision: 0.8113 - val_Recall: 0.7818 - val_custom_f1: 0.7947 - val_auc: 0.9506 - val_accuracy: 0.8000
Epoch 19/20
28/28 [==============================] - 2s 57ms/step - loss: 0.0862 - Precision: 0.9977 - Recall: 0.

In [24]:
y_pred = model.predict(X_test)

4/4 [==============================] - 2s 46ms/step


In [25]:
y_pred_clean = onehot_encoder.inverse_transform(y_pred)

In [26]:
y_pred_clean.tofile("EmotioNet_Emotion_Predictions.csv")

In [27]:
y_pred_clean = y_pred_clean.reshape(110)

In [31]:
y_test = y_test.reshape(110)

In [32]:
testing_results = pd.DataFrame({"Predicted value":y_pred_clean, "Real value": y_test})

In [34]:
testing_results.to_csv("./Results/EmotioNet_Emotion_Results.csv")

In [79]:
testing_results.to_pickle("./Final_Emotion_Results")

: 

In [35]:
from sklearn.metrics import confusion_matrix

result = confusion_matrix(y_test,y_pred_clean)

In [36]:
result

array([[ 2,  0,  1,  1,  1,  1],
       [ 0,  1,  0,  1,  5,  0],
       [ 0,  0,  2,  0,  3,  0],
       [ 0,  0,  0,  1,  1,  2],
       [ 0,  0,  0,  0, 72,  3],
       [ 2,  0,  1,  0,  3,  7]])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

pres